# Student Project: Pharmacovigilance Signal Detector

## 4-Agent AI System for Drug Safety Analysis

**📖 Complete Instructions**: See the project README.md for comprehensive project instructions including:
- Your mission and objectives
- Success criteria and requirements
- Getting started guide
- Implementation notes

### **Quick Start**
1. **Review README.md** for complete project context
2. **Complete STUDENT TODO sections** throughout this notebook
3. **Implement 4 specialized AI agents** with expert prompts
4. **Test your system** with different drug-adverse event combinations

### **What You'll Build**

A complete **4-agent AI system** that analyzes **FDA adverse event data (FAERS)** to detect potential drug safety signals:

#### **Agent Workflow**
```
Statistical Agent → Clinical Agent → Regulatory Agent → Quality Control → Final Report
```

#### **Implementation Tasks**
- **Statistical Agent**: Senior FDA biostatistician with ROR/PRR calculations
- **Clinical Agent**: Pharmacovigilance director with literature review capabilities  
- **Regulatory Agent**: FDA regulatory specialist with compliance reporting
- **Quality Control Agent**: QA director with feedback loop implementation

### **Learning Objectives**
This project demonstrates all 8 key concepts from the Life Science Agentic AI course through practical implementation.

## Part 1: Environment Setup & Data Loading

This section configures the necessary libraries and loads the FDA FAERS dataset for our 4-agent pharmacovigilance system.

### What We'll Configure:
- **Core Libraries**: pandas, numpy for data analysis
- **AI Framework**: LangChain for multi-agent coordination
- **OpenAI Integration**: GPT-4o-mini for intelligent agents
- **FAERS Data**: FDA adverse event reporting dataset

In [ ]:
# =============================================================================
# LIBRARY IMPORTS & SETUP
# =============================================================================

# Data Analysis Libraries
import pandas as pd
import numpy as np
from datetime import datetime
import json
from typing import List, Dict, Any, Optional
import os

# Environment Configuration
from dotenv import load_dotenv

# Custom Pharmacovigilance Tools
import pharma_tools

# LangChain Framework
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage


# Data Validation Framework
from pydantic import BaseModel, Field

print("All libraries imported successfully")

### AI Configuration

**📖 Setup Instructions**: See the project README.md for complete AI configuration details including:
- Vocareum API setup for Udacity students
- Direct OpenAI API configuration
- Environment file (.env) setup guide
- Project structure and file locations
- Troubleshooting tips

**Quick Setup**: Create a `.env` file in the project directory with your API credentials and run the configuration cells below.

In [ ]:
# =============================================================================
# OPENAI API CONFIGURATION
# =============================================================================

# Clear any cached environment variables and reload
import os
if 'OPENAI_API_KEY' in os.environ:
    del os.environ['OPENAI_API_KEY']
if 'BASE_URL' in os.environ:
    del os.environ['BASE_URL']

# Load environment variables fresh
load_dotenv(override=True)  # Force reload
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
Base_URL = os.getenv('BASE_URL')

if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key is required. Please create a .env file with OPENAI_API_KEY=your-key-here")

# Configure Language Model
llm = ChatOpenAI(
    model="gpt-4o-mini",          # Fast, efficient model
    temperature=0.1,              # Low randomness for consistent analysis  
    max_completion_tokens=2000,    # Sufficient for detailed responses
    base_url=Base_URL
)

print("OpenAI GPT-4o-mini configured successfully")
print("Multi-agent system ready for intelligent analysis")

In [ ]:
# =============================================================================
# TEST LLM CONNECTION & DEBUG API KEY
# =============================================================================

# Debug: Check which API key is being loaded
print("API Configuration Debug:")
print(f"API Key starts with: {OPENAI_API_KEY[:15]}...")
print(f"Base URL: {Base_URL}")
print(f"Is Vocareum key: {'voc-' in OPENAI_API_KEY}")

# Simple test to verify the LLM is working correctly
try:
    test_response = llm.invoke("Say 'API connection successful' if you can read this message.")
    print("\nLLM Connection Test:")
    print(f"Response: {test_response.content}")
    print("LLM is working correctly with your Vocareum API key!")
except Exception as e:
    print("\nLLM Test Failed:")
    print(f"Error: {str(e)}")
    if "Insufficient budget" in str(e):
        print("This suggests your Vocareum account has reached its usage limit.")
        print("   Contact your instructor or check your Vocareum dashboard for quota information.")
    else:
        print("Please check your API configuration above.")

In [ ]:
# =============================================================================
# FAERS DATA LOADING
# =============================================================================

data_path = "../data/faers_signal_detection_dataset.csv"

try:
    faers_data = pd.read_csv(data_path)
    print(f"FAERS dataset loaded successfully")
    print(f"Dataset: {len(faers_data):,} reports | {faers_data['drug_name'].nunique():,} drugs | {faers_data['adverse_event'].nunique():,} events")
    
    # Show top drugs by report volume
    print(f"\nTop 5 Most Reported Drugs:")
    top_drugs = faers_data['drug_name'].value_counts().head()
    for drug, count in top_drugs.items():
        print(f"   • {drug}: {count:,} reports")
        
except FileNotFoundError:
    raise FileNotFoundError(f"FAERS data not found at {data_path}. Please run: python download_data_demo.py")

## Part 2: Pharmacovigilance Tools Setup

Configure the specialized tools that our AI agents will use to analyze drug safety data.

### **STUDENT LEARNING OBJECTIVE**: 
Learn to select appropriate tools for each specialized agent role based on their expertise and responsibilities.

### **Available Tool Categories**:
- **Statistical Tools**: Data exploration, statistical calculations, result validation
- **Clinical Tools**: Literature research, biological mechanism assessment  
- **Regulatory Tools**: Compliance reporting, structured documentation
- **Quality Tools**: Cross-validation, consistency checking, feedback analysis

In [ ]:
# =============================================================================
# PHARMACOVIGILANCE TOOLS CONFIGURATION & DISCOVERY
# =============================================================================

# Reload tools module to get latest functions
import importlib
importlib.reload(pharma_tools)

# Connect FAERS data to tools
pharma_tools.set_faers_data(faers_data)

# Get all available tools for agents
pharma_tools_list = pharma_tools.get_all_pharma_tools()

# =============================================================================
# TOOL CATEGORIZATION FOR STUDENT LEARNING
# =============================================================================

print("AVAILABLE PHARMACOVIGILANCE TOOLS BY CATEGORY:")
print("=" * 70)

# Categorize tools for educational purposes
available_statistical_tools = []
available_clinical_tools = []
available_regulatory_tools = []
available_quality_tools = []

for tool in pharma_tools_list:
    tool_name = tool.name
    tool_desc = tool.description
    
    # Smart categorization based on function purpose and keywords
    if any(keyword in tool_name.lower() for keyword in ['drug', 'statistic', 'calculate', 'validate']):
        available_statistical_tools.append(tool)
        category = "STATISTICAL"
    elif any(keyword in tool_name.lower() for keyword in ['clinical', 'literature', 'biological', 'plausibility']):
        available_clinical_tools.append(tool)
        category = "CLINICAL"
    elif any(keyword in tool_name.lower() for keyword in ['regulatory', 'report', 'generate']):
        available_regulatory_tools.append(tool)
        category = "REGULATORY"
    elif any(keyword in tool_name.lower() for keyword in ['quality', 'review', 'analysis']):
        available_quality_tools.append(tool)
        category = "QUALITY"
    else:
        available_statistical_tools.append(tool)  # Default to statistical
        category = "STATISTICAL"
    
    print(f"{category}: {tool_name}")
    print(f"   {tool_desc}")
    print()

print("TOOL SUMMARY:")
print(f"   Statistical Tools: {len(available_statistical_tools)}")
print(f"   Clinical Tools: {len(available_clinical_tools)}")  
print(f"   Regulatory Tools: {len(available_regulatory_tools)}")
print(f"   Quality Tools: {len(available_quality_tools)}")
print(f"   Total Available: {len(pharma_tools_list)}")

print("\nAll tools categorized and ready for agent assignment!")

## Tool Selection Exercise - **STUDENT REQUIRED**

**LEARNING OBJECTIVE**: Based on the tool exploration above, you must now define which specific tools each agent should use.

**YOUR TASK**: Complete the tool assignments below by selecting the most appropriate tools for each agent role:

- **Statistical Agent** (FDA Biostatistician): Tools for data analysis, calculations, validation
- **Clinical Agent** (Pharmacovigilance Director): Tools for research, clinical assessment  
- **Regulatory Agent** (FDA Regulatory Specialist): Tools for report generation, compliance
- **Quality Control Agent** (QA Director): Tools for review, validation, feedback

**Requirements**:
1. Select 2-4 tools per agent based on their expertise
2. Provide clear rationale for each selection
3. Consider the agent's real-world responsibilities

In [ ]:
# =============================================================================
# STUDENT TODO: DEFINE TOOL LISTS FOR EACH AGENT
# =============================================================================

# TODO: Based on the tool exploration above, define which tools each agent should use
# Look at the categorized tools printed above and select the most appropriate ones

# Statistical Agent Tools
# TODO: List the tools the Statistical Agent should have access to
# Consider: What statistical analysis capabilities does this agent need?
statistical_agent_tools = [
    # TODO: Add your selected tools here
    # Example format: pharma_tools.tool_name_1, pharma_tools.tool_name_2, ...
]

# Clinical Agent Tools  
# TODO: List the tools the Clinical Agent should have access to
# Consider: What clinical knowledge and analysis tools are needed?
clinical_agent_tools = [
    # TODO: Add your selected tools here
    # Example format: pharma_tools.tool_name_1, pharma_tools.tool_name_2, ...
]

# Regulatory Agent Tools
# TODO: List the tools the Regulatory Agent should have access to
# Consider: What regulatory compliance tools are essential?
regulatory_agent_tools = [
    # TODO: Add your selected tools here
    # Example format: pharma_tools.tool_name_1
]

# Quality Control Agent Tools
# TODO: List the tools the Quality Control Agent should have access to
# Consider: What validation and verification tools are needed?
quality_control_agent_tools = [
    # TODO: Add your selected tools here
    # Example format: pharma_tools.tool_name_1
]

# Display your selections
print("STUDENT TOOL SELECTIONS:")
print("=" * 50)
print(f"Statistical Agent: {len(statistical_agent_tools)} tools selected")
print(f"Clinical Agent: {len(clinical_agent_tools)} tools selected")
print(f"Regulatory Agent: {len(regulatory_agent_tools)} tools selected")
print(f"Quality Control Agent: {len(quality_control_agent_tools)} tools selected")

if all([statistical_agent_tools, clinical_agent_tools, regulatory_agent_tools, quality_control_agent_tools]):
    print("\n✓ All agents have tools assigned!")
    print("Ready to proceed to agent creation!")
else:
    print("\n⚠ Please complete tool selection for all agents before proceeding.")

## Part 3: AI Agent Creation - **STUDENT IMPLEMENTATION REQUIRED**

**YOUR MAIN TASKS**: 
1. **Select appropriate tools** for each agent based on their expertise
2. **Complete agent system prompts** following detailed instructions

### **Learning Objective**: 
Understand how to match specialist tools to agent roles and create professional system prompts.

**📖 Detailed Reference**: See the project README.md for comprehensive information about:
- Complete agent personas and expertise areas
- Detailed tool selection strategy
- Agent workflow architectures  
- Real-world role equivalents

### **Quick Agent Overview**:

- **Statistical Agent** (FDA Biostatistician): Data analysis, calculations, validation tools
- **Clinical Agent** (Pharmacovigilance Director): Research, clinical assessment tools
- **Regulatory Agent** (FDA Regulatory Specialist): Report generation, compliance tools
- **Quality Control Agent** (QA Director): Review, validation, feedback tools


### **Requirements**:

1. **All 4 agents** must have implemented prompts (no placeholders)
2. Each prompt must follow the detailed instructions provided
3. Agents must use their assigned tools appropriately
4. Professional pharmaceutical industry language required
5. Step-by-step reasoning workflows mandatory

### Agent 1: Statistical Analysis Agent

**Role**: Senior FDA Biostatistician  
**Expertise**: Signal detection, ROR/PRR calculations, statistical validation  
**Tools**: Drug statistics calculator, results validator, data explorer

In [ ]:
# =============================================================================
# STATISTICAL ANALYSIS AGENT CREATION - STUDENT TODO
# =============================================================================

# =============================================================================
# STUDENT TODO #1: Use Your Selected Tools from Tool Selection Exercise Above
# =============================================================================

# Use the tools you selected in the Tool Selection Exercise above
statistical_tools = statistical_agent_tools  # Reference to your tool selection above

# Create modern LangChain agent with tool binding (only if tools are selected)
if statistical_tools:
    statistical_agent = llm.bind_tools(statistical_tools)
else:
    statistical_agent = None

# =============================================================================
# STUDENT TODO: Create Statistical Agent System Prompt
# =============================================================================

# TODO: Create a professional system prompt for a Senior FDA Biostatistician
# STUDENT TODO: Write your Statistical Agent system prompt below
# Create an expert FDA biostatistician persona with appropriate expertise and workflow
statistical_system_prompt = """

"""

print("Statistical Analysis Agent created successfully!")
print("STUDENT: Complete the system prompt above to define the agent's expertise")

### Agent 2: Clinical Assessment Agent

**Role**: Director of Pharmacovigilance (20+ years experience)  
**Expertise**: Clinical interpretation, biological plausibility, risk assessment  
**Tools**: Literature search engine, mechanism assessment analyzer

In [ ]:
# =============================================================================
# CLINICAL ASSESSMENT AGENT CREATION - STUDENT TODO
# =============================================================================

# =============================================================================
# STUDENT TODO #2: Use Your Selected Tools from Tool Selection Exercise Above
# =============================================================================

# Use the tools you selected in the Tool Selection Exercise above
clinical_tools = clinical_agent_tools  # Reference to your tool selection above

# Create modern LangChain agent with tool binding (only if tools are selected)
if clinical_tools:
    clinical_agent = llm.bind_tools(clinical_tools)
else:
    clinical_agent = None

# =============================================================================
# STUDENT TODO: Create Clinical Assessment Agent System Prompt
# =============================================================================

# TODO: Create a professional system prompt for a Director of Pharmacovigilance
# STUDENT TODO: Write your Clinical Assessment Agent system prompt below
# Create an expert pharmacovigilance director persona with appropriate expertise and workflow
clinical_system_prompt = """

"""

print("Clinical Assessment Agent created successfully!")
print("STUDENT: Complete the system prompt above to define the agent's clinical expertise")

### Agent 3: Regulatory Reporter Agent

**Role**: FDA Regulatory Affairs Specialist (15+ years experience)  
**Expertise**: FDA compliance, ICH guidelines, structured reporting  
**Tools**: Regulatory report generator with FDA-compliant formatting

In [ ]:
# =============================================================================
# REGULATORY REPORTER AGENT CREATION - STUDENT TODO
# =============================================================================

# =============================================================================
# STUDENT TODO #3: Use Your Selected Tools from Tool Selection Exercise Above
# =============================================================================

# Use the tools you selected in the Tool Selection Exercise above
regulatory_tools = regulatory_agent_tools  # Reference to your tool selection above

# Create modern LangChain agent with tool binding (only if tools are selected)
if regulatory_tools:
    regulatory_agent = llm.bind_tools(regulatory_tools)
else:
    regulatory_agent = None

# =============================================================================
# STUDENT TODO: Create Regulatory Reporter Agent System Prompt
# =============================================================================

# TODO: Create a professional system prompt for an FDA Regulatory Affairs Specialist
# STUDENT TODO: Write your Regulatory Reporter Agent system prompt here
# Create an expert FDA regulatory affairs specialist persona with appropriate expertise and workflow
regulatory_system_prompt = """
"""

print("Regulatory Reporter Agent created successfully!")
print("STUDENT: Complete the system prompt above to define the agent's regulatory expertise")

### Agent 4: Quality Control Agent (Feedback Loop)

**Role**: Senior Director of Pharmacovigilance QA (20+ years experience)  
**Expertise**: Cross-validation, feedback loops, consistency analysis  
**Tools**: Quality review analyzer with feedback loop implementation

In [ ]:
# =============================================================================
# QUALITY CONTROL AGENT CREATION - STUDENT TODO
# =============================================================================

# =============================================================================
# STUDENT TODO #4: Use Your Selected Tools from Tool Selection Exercise Above
# =============================================================================

# Use the tools you selected in the Tool Selection Exercise above
qc_tools = quality_control_agent_tools  # Reference to your tool selection above

# Create modern LangChain agent with tool binding (only if tools are selected)
if qc_tools:
    qc_agent = llm.bind_tools(qc_tools)
else:
    qc_agent = None

# =============================================================================
# STUDENT TODO: Create Quality Control Agent System Prompt
# =============================================================================

# TODO: Create a professional system prompt for a Senior Director of Pharmacovigilance QA
qc_system_prompt = """
# STUDENT TODO: Write your Quality Control Agent system prompt here
# Create an expert QA director persona with appropriate expertise and workflow
"""

print("Quality Control Agent created successfully!")
print("STUDENT: Complete the system prompt above to define the agent's QA expertise")

In [ ]:
# =============================================================================
# MULTI-AGENT SYSTEM STATUS CHECK
# =============================================================================

# Check both tool selection and agent creation
print("STUDENT PROGRESS CHECK:")
print("=" * 50)

# Check tool selection status
tool_status = {
    "Statistical Tools": len(statistical_tools) > 0,
    "Clinical Tools": len(clinical_tools) > 0,
    "Regulatory Tools": len(regulatory_tools) > 0,
    "Quality Control Tools": len(qc_tools) > 0
}

# Check agent creation status  
agents_status = {
    "Statistical Agent": statistical_agent is not None,
    "Clinical Agent": clinical_agent is not None,
    "Regulatory Agent": regulatory_agent is not None,
    "Quality Control Agent": qc_agent is not None
}

print("TOOL SELECTION STATUS:")
for category, status in tool_status.items():
    status_icon = "✓" if status else "⚠"
    print(f"   {status_icon} {category}")

print("\nAGENT CREATION STATUS:")
for agent_name, status in agents_status.items():
    status_icon = "✓" if status else "✗"
    print(f"   {status_icon} {agent_name}")

successful_tools = sum(tool_status.values())
successful_agents = sum(agents_status.values())
total_components = len(tool_status)

print(f"\nOVERALL PROGRESS:")
print(f"   Tools Selected: {successful_tools}/{total_components}")
print(f"   Agents Created: {successful_agents}/{total_components}")

if successful_tools == total_components and successful_agents == total_components:
    print("\nEXCELLENT! Complete multi-agent system ready!")
    print("Workflow: Statistical → Clinical → Regulatory → Quality Control")
    print("Ready for Part 4: Multi-Agent Analysis!")
elif successful_tools < total_components:
    print(f"\nSTUDENT TODO: Complete tool selection for all {total_components} agents")
    print("Tip: Review the available tools printed above and select appropriate ones")
else:
    print(f"\nSTUDENT TODO: Complete system prompt implementation")
    print("Tip: Each agent needs both tools AND a professional system prompt")

## Part 4: Multi-Agent Pharmacovigilance Analysis

This section demonstrates our complete 4-agent system analyzing a real-world drug safety signal.

### Complete Analysis Workflow:

1. **Statistical Agent**: Calculates ROR/PRR, validates statistical significance
2. **Clinical Agent**: Assesses biological plausibility, reviews literature  
3. **Regulatory Agent**: Generates FDA-compliant safety report
4. **Quality Control Agent**: Reviews all findings, provides feedback for improvement
5. **Final Integration**: Comprehensive recommendations based on all agent inputs

In [ ]:
# =============================================================================
# COMPLETE 4-AGENT PHARMACOVIGILANCE ANALYSIS FUNCTION
# =============================================================================

def analyze_drug_safety_signal(drug_name: str, adverse_event: str) -> Dict[str, Any]:
    """
    Complete 4-agent pharmacovigilance analysis with feedback loops
    
    Workflow:
    1. Statistical Agent: Calculate ROR/PRR statistics 
    2. Clinical Agent: Assess biological plausibility
    3. Regulatory Agent: Generate FDA-compliant report
    4. Quality Control Agent: Review for consistency and provide feedback
    5. Final Integration: Comprehensive recommendations
    """
    print(f"ANALYZING: {drug_name} → {adverse_event}")
    print("=" * 60)
    
    analysis_results = {
        "drug_name": drug_name,
        "adverse_event": adverse_event,
        "timestamp": datetime.now().isoformat(),
        "statistical_analysis": None,
        "clinical_assessment": None, 
        "regulatory_report": None,
        "quality_review": None,
        "final_recommendation": None
    }
    
    # Variables to store raw tool outputs for cross-agent use
    raw_statistical_data = None
    raw_clinical_data = {}
            
    # =================================================================
    # STEP 1: STATISTICAL ANALYSIS AGENT
    # =================================================================
    print("STEP 1: Statistical Analysis")
    print("-" * 30)
    
    statistical_query = f"""
    Please perform a complete statistical analysis for the drug-adverse event pair:
    Drug: {drug_name}
    Adverse Event: {adverse_event}
    
    Please:
    1. Calculate ROR and PRR statistics using the appropriate tool
    2. Validate the calculations for accuracy
    3. Interpret the results according to FDA/EMA guidelines
    4. Determine if a statistical signal is detected
    """
    
    messages = [
        SystemMessage(content=statistical_system_prompt),
        HumanMessage(content=statistical_query)
    ]
    
    statistical_result = statistical_agent.invoke(messages)
    
    if hasattr(statistical_result, 'tool_calls') and statistical_result.tool_calls:
        messages.append(statistical_result)
        tool_result_str = ""
        for tool_call in statistical_result.tool_calls:
            tool_to_call = next(t for t in statistical_tools if t.name == tool_call["name"])
            tool_output = tool_to_call.invoke(tool_call["args"])
            
            if tool_call["name"] == "calculate_drug_event_statistics":
                raw_statistical_data = tool_output

            tool_result_str = str(tool_output)
            messages.append(ToolMessage(content=tool_result_str, tool_call_id=tool_call["id"]))

        final_result = statistical_agent.invoke(messages)
        final_content = final_result.content if final_result.content and final_result.content.strip() else tool_result_str
        
        analysis_results["statistical_analysis"] = {
            "agent_response": final_content,
            "tool_usage": [tc["name"] for tc in statistical_result.tool_calls],
            "raw_data": tool_result_str,
            "status": "completed"
        }
    else:
        analysis_results["statistical_analysis"] = {
            "agent_response": statistical_result.content,
            "tool_usage": [], "raw_data": "", "status": "completed"
        }

    # =================================================================
    # STEP 2: CLINICAL ASSESSMENT AGENT
    # =================================================================
    print("\nSTEP 2: Clinical Assessment")
    print("-" * 30)
    
    statistical_response = analysis_results.get("statistical_analysis", {}).get("agent_response", "")
    
    clinical_query = f"""
    Based on the statistical analysis, provide a clinical assessment for:
    Drug: {drug_name}, Adverse Event: {adverse_event}
    Statistical Context: {statistical_response}
    """
    
    messages = [SystemMessage(content=clinical_system_prompt), HumanMessage(content=clinical_query)]
    clinical_result = clinical_agent.invoke(messages)
    
    if hasattr(clinical_result, 'tool_calls') and clinical_result.tool_calls:
        messages.append(clinical_result)
        tool_result_str = ""
        for tool_call in clinical_result.tool_calls:
            tool_to_call = next(t for t in clinical_tools if t.name == tool_call["name"])
            tool_output = tool_to_call.invoke(tool_call["args"])
            
            raw_clinical_data[tool_call["name"]] = tool_output
            
            tool_result_str += str(tool_output) + "\n"
            messages.append(ToolMessage(content=str(tool_output), tool_call_id=tool_call["id"]))

        final_result = clinical_agent.invoke(messages)
        final_content = final_result.content if final_result.content and final_result.content.strip() else tool_result_str
        
        analysis_results["clinical_assessment"] = {
            "agent_response": final_content,
            "tool_usage": [tc["name"] for tc in clinical_result.tool_calls],
            "raw_data": tool_result_str, "status": "completed"
        }
    else:
        analysis_results["clinical_assessment"] = {
            "agent_response": clinical_result.content,
            "tool_usage": [], "raw_data": "", "status": "completed"
        }

    # =================================================================
    # STEP 3: REGULATORY REPORTER AGENT
    # =================================================================
    print("\nSTEP 3: Regulatory Report")
    print("-" * 30)
    
    clinical_response = analysis_results.get("clinical_assessment", {}).get("agent_response", "")

    # Prepare data for regulatory tool
    combined_clinical_data = {}
    
    if "search_clinical_literature" in raw_clinical_data:
        lit_data = json.loads(raw_clinical_data["search_clinical_literature"]) if isinstance(raw_clinical_data["search_clinical_literature"], str) else raw_clinical_data["search_clinical_literature"]
        combined_clinical_data.update(lit_data)
    
    if "assess_biological_plausibility" in raw_clinical_data:
        plaus_data = json.loads(raw_clinical_data["assess_biological_plausibility"]) if isinstance(raw_clinical_data["assess_biological_plausibility"], str) else raw_clinical_data["assess_biological_plausibility"]
        combined_clinical_data.update(plaus_data)
    
    report_args = {
        "drug_name": drug_name,
        "adverse_event": adverse_event,
        "statistical_data": raw_statistical_data or '{"error": "missing data"}',
        "clinical_assessment": json.dumps(combined_clinical_data) if combined_clinical_data else '{"error": "missing data"}'
    }
    
    tool_output = pharma_tools.generate_regulatory_report.invoke(report_args)
    
    analysis_results["regulatory_report"] = {
        "agent_response": str(tool_output),
        "tool_usage": ["generate_regulatory_report"],
        "raw_data": str(tool_output),
        "status": "completed"
    }

    # =================================================================
    # STEP 4: QUALITY CONTROL AGENT
    # =================================================================
    print("\nSTEP 4: Quality Control & Feedback")
    print("-" * 30)
    
    regulatory_response = analysis_results.get("regulatory_report", {}).get("agent_response", "")

    qc_query = f"""
    Please perform a comprehensive quality control review of the complete pharmacovigilance analysis for:
    Drug: {drug_name}
    Adverse Event: {adverse_event}
    
    Agent Findings Summary:
    
    STATISTICAL ANALYSIS:
    {statistical_response[:500]}...
    
    CLINICAL ASSESSMENT:
    {clinical_response[:500]}...
    
    REGULATORY REPORT:
    {regulatory_response[:500]}...
    
    Your task:
    1. Use the quality_review_analysis tool to systematically review all findings
    2. Identify any inconsistencies between agent conclusions
    3. Check alignment between statistical signals and clinical plausibility
    4. Verify regulatory risk assessments match evidence strength
    5. Provide specific feedback for any areas needing improvement
    6. Make final approval recommendations
    """
    
    messages = [SystemMessage(content=qc_system_prompt), HumanMessage(content=qc_query)]
    qc_result = qc_agent.invoke(messages)
    
    if hasattr(qc_result, 'tool_calls') and qc_result.tool_calls:
        messages.append(qc_result)
        tool_result_str = ""
        for tool_call in qc_result.tool_calls:
            tool_to_call = next(t for t in qc_tools if t.name == tool_call["name"])
            
            if tool_call["name"] == "quality_review_analysis":
                statistical_data_for_qc = raw_statistical_data or '{"error": "missing statistical data"}'
                
                combined_clinical_for_qc = {}
                if "search_clinical_literature" in raw_clinical_data:
                    lit_data = json.loads(raw_clinical_data["search_clinical_literature"]) if isinstance(raw_clinical_data["search_clinical_literature"], str) else raw_clinical_data["search_clinical_literature"]
                    combined_clinical_for_qc.update(lit_data)
                
                if "assess_biological_plausibility" in raw_clinical_data:
                    plaus_data = json.loads(raw_clinical_data["assess_biological_plausibility"]) if isinstance(raw_clinical_data["assess_biological_plausibility"], str) else raw_clinical_data["assess_biological_plausibility"]
                    combined_clinical_for_qc.update(plaus_data)
                
                clinical_data_for_qc = json.dumps(combined_clinical_for_qc) if combined_clinical_for_qc else '{"error": "missing clinical data"}'
                regulatory_data_for_qc = analysis_results.get("regulatory_report", {}).get("raw_data", '{"error": "missing regulatory data"}')
                
                tool_args = {
                    "statistical_findings": statistical_data_for_qc,
                    "clinical_findings": clinical_data_for_qc,
                    "regulatory_report": regulatory_data_for_qc
                }
            else:
                tool_args = tool_call["args"]
            
            tool_output = tool_to_call.invoke(tool_args)
            tool_result_str = str(tool_output)
            messages.append(ToolMessage(content=tool_result_str, tool_call_id=tool_call["id"]))

        final_result = qc_agent.invoke(messages)
        final_content = final_result.content if final_result.content and final_result.content.strip() else tool_result_str

        analysis_results["quality_review"] = {
            "agent_response": final_content,
            "tool_usage": [tc["name"] for tc in qc_result.tool_calls],
            "raw_data": tool_result_str, "status": "completed"
        }
    else:
        analysis_results["quality_review"] = {
            "agent_response": qc_result.content,
            "tool_usage": [], "raw_data": "", "status": "completed"
        }

    # =================================================================
    # STEP 5: FINAL INTEGRATION & RECOMMENDATIONS
    # =================================================================
    print("\nSTEP 5: Final Integration")
    print("-" * 30)
    
    workflow_completed = []
    for k, v in analysis_results.items():
        if k in ["statistical_analysis", "clinical_assessment", "regulatory_report", "quality_review"]:
            if v and isinstance(v, dict) and v.get("status") == "completed":
                workflow_completed.append(k)
    
    analysis_results["final_recommendation"] = {
        "recommendation": "ANALYSIS COMPLETED - Comprehensive pharmacovigilance assessment with multi-agent validation",
        "reasoning": f"Successfully completed {len(workflow_completed)}/4 workflow stages",
        "workflow_completed": workflow_completed,
        "next_steps": [
            "Review all agent findings for comprehensive understanding",
            "Consider clinical consultation if warranted",
            "Monitor for additional safety signals",
            "Update safety database with findings"
        ]
    }
    
    print("Complete 4-agent analysis finished!")
    
    return analysis_results

print("Complete 4-agent pharmacovigilance analysis function ready!")
print("Multi-agent workflow: Statistical → Clinical → Regulatory → Quality Control")
print("Ready to analyze drug safety signals!")

## Part 5: Interactive Drug Safety Analysis

### Configurable Drug Safety Analysis

**How to Use This Cell:**
- Simply modify the `drug_name` and `adverse_event` variables below
- Run the cell to see the complete 4-agent analysis in action

**📖 Example Combinations**: See the project README.md for detailed examples including:
- Well-documented associations (strong signals)
- Common side effects (moderate signals)  
- Educational value of different signal strengths

**What You'll See:**
1. **Statistical Analysis**: ROR/PRR calculations and signal detection
2. **Clinical Assessment**: Biological plausibility and literature review
3. **Regulatory Report**: FDA-compliant safety documentation  
4. **Quality Control**: Multi-agent consistency validation
5. **Final Integration**: Comprehensive recommendations

In [ ]:
# =============================================================================
# CONFIGURABLE DRUG SAFETY ANALYSIS
# =============================================================================

# CONFIGURE YOUR ANALYSIS HERE - Change these variables to test any drug combination
drug_name = "METFORMIN"                    # Change to any drug name
adverse_event = "LACTIC ACIDOSIS"          # Change to any adverse event

# Alternative examples you can try (just uncomment one):
# drug_name, adverse_event = "WARFARIN", "BLEEDING"
# drug_name, adverse_event = "ASPIRIN", "GASTROINTESTINAL HEMORRHAGE"  
# drug_name, adverse_event = "SIMVASTATIN", "MYOPATHY"
# drug_name, adverse_event = "ACETAMINOPHEN", "HEPATOTOXICITY"

print(f"COMPLETE 4-AGENT ANALYSIS: {drug_name} & {adverse_event}")
print("This demonstrates the full pharmacovigilance workflow with feedback loops\n")

# Run the complete analysis
results = analyze_drug_safety_signal(drug_name, adverse_event)

# Display all agent findings
print("\n" + "="*60)
print("STEP 1: STATISTICAL FINDINGS")
print("="*60)
if results["statistical_analysis"]:
    print("Tools used:", ", ".join(results["statistical_analysis"]["tool_usage"]))
    print("\nStatistical Analysis:")
    print(results["statistical_analysis"]["agent_response"])

print("\n" + "="*60)
print("STEP 2: CLINICAL ASSESSMENT")
print("="*60) 
if results["clinical_assessment"]:
    print("Tools used:", ", ".join(results["clinical_assessment"]["tool_usage"]))
    print("\nClinical Assessment:")
    print(results["clinical_assessment"]["agent_response"])

print("\n" + "="*60)
print("STEP 3: REGULATORY REPORT")
print("="*60)
if results["regulatory_report"]:
    print("Tools used:", ", ".join(results["regulatory_report"]["tool_usage"]))
    print("\nRegulatory Report:")
    print(results["regulatory_report"]["agent_response"])

print("\n" + "="*60)
print("STEP 4: QUALITY CONTROL & FEEDBACK")
print("="*60)
if results["quality_review"]:
    print("Tools used:", ", ".join(results["quality_review"]["tool_usage"]))
    print("\nQuality Review:")
    print(results["quality_review"]["agent_response"])

print("\n" + "="*60)
print("STEP 5: FINAL RECOMMENDATION")
print("="*60)
if results["final_recommendation"]:
    print(f"Recommendation: {results['final_recommendation']['recommendation']}")
    print(f"Reasoning: {results['final_recommendation']['reasoning']}")
    print("Workflow Completed:", ", ".join(results['final_recommendation']['workflow_completed']))
    print("Next Steps:")
    for step in results['final_recommendation']['next_steps']:
        print(f"  • {step}")

print(f"\nComplete 4-agent pharmacovigilance analysis finished!")
print(f"TIP: Change the drug_name and adverse_event variables above to test other combinations!")

## Part 6: Project Summary & Learning Objectives

### What You've Accomplished

Congratulations! You've successfully built and demonstrated a complete **4-agent pharmacovigilance system** that showcases all key concepts from the Life Science Agentic AI course:

#### **All 8 Learning Objectives Achieved:**
1. **Role-Based Prompting**: 4 expert AI personas with specialized knowledge
2. **Chain-of-Thought Reasoning**: Structured analysis workflows for each agent
3. **Multi-Agent Systems**: Coordinated workflow with LangChain framework
4. **Statistical Signal Detection**: ROR/PRR calculations with FDA guidelines
5. **Validation Mechanisms**: Tools prevent AI hallucination of data
6. **Regulatory Reporting**: FDA-compliant safety report generation
7. **Feedback Loops**: Quality control agent with consistency checking
8. **Real-World Application**: Authentic FAERS data analysis